In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import logging 
import logging.config
import shutil
import sys
from itertools import cycle
from inspect import getfullargspec
from importlib import import_module
from time import sleep
from pathlib import Path
from distutils.util import strtobool
import waveshare_epd

In [3]:
import ArgConfigParse
from epdlib import Screen
from epdlib.Screen import Update
from epdlib.Screen import ScreenError
from library.CacheFiles import CacheFiles
from library.Plugin import Plugin
from library.InterruptHandler import InterruptHandler
from library import get_help
from library import run_module
import my_constants as constants


In [4]:
def do_exit(status=0, message=None, **kwargs):
    if message:
        if status > 0:
            logging.error(f'failure caused exit: {message}')
        border = '\n'+'#'*70 + '\n'
        message = border + message + border + '\n***Exiting***'
        print(message)
        
    try:
        sys.exit(status)
    except Exception as e:
        pass

In [5]:
def clean_up(cache=None, screen=None):
    logging.info('cleaning up cache and screen')
    try:
        cache.cleanup()
    except AttributeError:
        logging.debug('no cache passed, skipping')
    try:
#         screen.initEPD()
        screen.clearEPD()
    except AttributeError:
        logging.debug('no screen passed, skipping')
    return

In [6]:
def get_cmd_line_args():
    cmd_args = ArgConfigParse.CmdArgs()
    cmd_args.add_argument('-c', '--config', ignore_none=True, metavar='CONFIG_FILE.ini',
                         type=str, dest='user_config',
                         help='use the specified configuration file')
    
    cmd_args.add_argument('-l', '--log_level', ignore_none=True, metavar='LOG_LEVEL',
                         type=str, choices=['DEBUG', 'INFO', 'WARNING', 'ERROR'],
                         dest='main__log_level', help='change the log output level')
    
    cmd_args.add_argument('--plugin_info', metavar='[plugin|plugin.function]',
                         required=False, default=None,
                         ignore_none=True,
                         help='get information for plugins and user-facing functions provided by a plugin')
    
    cmd_args.add_argument('--list_plugins', required=False,
                         default=False, action='store_true', 
                         help='list all available plugins')
    
    cmd_args.add_argument('--run_plugin_func',
                         required=False, default=None, nargs='+',
                         metavar=('plugin.function', 'optional_arg1 arg2 argN'),
                         ignore_none=True,
                         help='run a user-facing function for a plugin')
    
    cmd_args.add_argument('-d', '--daemon', required=False, default=False,
                         dest='main__daemon', action='store_true', 
                         help='run in daemon mode (ignore user configuration if found)')
    
    cmd_args.add_argument('-R', '--max_refresh', required=False, default=3, 
                          dest='main__max_refresh',
                          help='maximum number of refreshes between complete screen refresh')    
    
    cmd_args.add_argument('-V', '--version', required=False, default=False, ignore_false=True,
                          action='store_true',
                          help='display version and exit')
    

    
   
    cmd_args.parse_args()    
 
    return cmd_args


In [7]:
def get_config_files(cmd_args):
    '''read config.ini style file(s)
    
    Args:
        cmd_args(`ArgConfigParse.CmdArgs` obj)
    
    Returns:
        ArgConfigParse.ConfigFile'''
    
    logging.debug('gathering configuration files')
    
    config_files_dict = {'base': constants.config_base,
                         'system': constants.config_system,
                         'user': constants.config_user,
                         'cmd_line': cmd_args.options.user_config}
    
    config_files_list = [config_files_dict['base']]
    
    if cmd_args.options.main__daemon:
        logging.debug(f'using daemon configuration: {constants.config_system}')
        config_files_list.append(config_files_dict['system'])
    else:
        if constants.config_user.exists():
            config_files_list.append(config_files_dict['user'])
        else:
            try:
                constants.config_user.parent.mkdir(parents=True, exist_ok=True)
            except PermissionError as e:
                msg=f'could not create user configuration directory: {constants.config_user.parent}'
                logging.critical(msg)
                do_exit(1, msg)
            try:
                shutil.copy(constants.config_base, constants.config_user)
            except Exception as e:
                msg=f'could not copy user configuration file to {constants.config_user}'
                logging.critical(1, msg)
                do_exit(1, msg)
            msg = f'''This appears to be the first time PaperPi has been run.
A user configuration file created: {constants.config_user}
At minimum you edit this file and add a display_type and enable one plugin.
        
Edit the configuration file with:
   $ nano {constants.config_user}'''
            do_exit(0, msg)
            
    
    
            
    logging.info(f'using configuration files to configure PaperPi: {config_files_list}')
    config_files = ArgConfigParse.ConfigFile(config_files_list, ignore_missing=True)
    config_files.parse_config()

    return config_files
        
        

In [8]:
def sanitize_vals(config):
    '''attempt to convert all the strings into appropriate formats
             integer/float like strings ('7', '100', '-1.3') -> int or float
             boolean like strings (yes, no, Y, t, f, on, off) -> 0 or 1
         Args:
             config(`dict`): nested config.ini style dictionary

         Returns:
             `dict`'''    
    def strtofloat(s):
        '''strings to float if possible'''
        retval = s
        if isinstance(s, str):
            if '.' in s:
                try:
                    retval = float(s)
                except ValueError:
                    pass

        return retval

    def convert(d, new_type, exceptions):
        for section, values in d.items():
            for key, value in values.items():
                if isinstance(value, str):
                    try:
                        sanitized = new_type(value)
                    except exceptions:
                        sanitized = value

                    d[section][key] = sanitized
                else:
                    d[section][key] = value
        return d

    convert(config, strtofloat, ValueError)
    convert(config, int, (ValueError))
    convert(config, strtobool, (ValueError, AttributeError))
    
    return config

In [9]:
def setup_splash(config, resolution):
    logging.debug('checking splash settings')
    if 'splash' in config['main']:
        logging.debug('checking splash screen settings')
        if config['main']['splash']:
            logging.debug('splash enabled in confg file')
            splash = True
        else:
            logging.debug('splash disabled in config file')
            splash = False
    else:
        splash = True

    if splash:
        logging.debug('splash screen enabled')
        from plugins.splash_screen import splash_screen
        splash_config = { 
            'name': 'Splash Screen',
            'layout': splash_screen.layout.layout,
            'update_function': splash_screen.update_function,
            'resolution': resolution
        } 
        logging.debug(f'configuring splash screen: {splash_config}')
        splash = Plugin(**splash_config)
        splash.update(constants.app_name, constants.version, constants.url)
        logging.debug(f'splash screen image type: {type(splash.image)}')
    return splash

In [10]:
def setup_display(config):
    def ret_obj(obj=None, status=0, message=None):
        return{'obj': obj, 'status': status, 'message': message}    
    keyError_fmt = 'configuration KeyError: section[{}], key: {}'

    moduleNotFoundError_fmt = 'could not load epd module: {} -- error: {}'
    
#     try:
#         logging.debug('setting display type')
#         epd_module = '.'.join([constants.waveshare_epd, config['main']['display_type']])
#         epd = import_module(epd_module)
#     except KeyError as e:
#         return_val = ret_obj(obj=None, status=1, message=keyError_fmt.format('main', 'display_type'))
#         logging.error(return_val['message'])
#         return return_val
#     except ModuleNotFoundError as e:
#         logging.error('Check your config files and ensure a known waveshare_epd display is specified!')
#         return_val = ret_obj(None, 1, moduleNotFoundError_fmt.format(config["main"]["display_type"], e))
#         return return_val
#     except FileNotFoundError as e:
#         msg = f''''Error loading waveshare_epd module: {e}
#         This is typically due to SPI not being enabled, or the current user is 
#         not a member of the SPI group.
#         "$ sudo raspi-config nonint get_spi" will return 0 if SPI is enabled
#         Try enabling SPI and run this program again. '''
#         logging.error(msg)
#         return_val = ret_obj(obj=None, status=1, message=msg)
#         return return_val
    
    epd = config['main']['display_type']
    vcom = config['main']['vcom']
    try:
        screen = Screen(epd=epd, vcom=vcom)
    except ScreenError as e:
        logging.critical('Error loading epd from configuration')
        return_val = ret_obj(None, 1, moduleNotFoundError_fmt.format(epd, e))
        return return_val
    except PermissionError as e:
        logging.critical(f'Error initializing EPD: {e}')
        logging.critical(f'The user executing {constants.app_name} does not have access to the SPI device.')
        return_val = ret_obj(None, 1, 'This user does not have access to the SPI group\nThis can typically be resolved by running:\n$ sudo groupadd <username> spi')
        return return_val
    except FileNotFoundError as e:
        logging.critical(f'Error initializing EPD: {e}')
        logging.critical(f'It appears that SPI is not enabled on this Pi. See: https://github.com/txoof/epd_display/tree/testing#hardwareos-setup')
        return_val = ret_obj(None, 1, moduleNotFoundError_fmt.format(epd, e))
        return return_val        

    try:
        config['main']['rotation'] = int(config['main']['rotation'])
    except KeyError as e:
        logging.info(keyError_fmt.format('main', 'rotation'))
        logging.info('using default: 0')
    try:
        screen.rotation = config['main']['rotation']
    except ValueError as e:
        logging.error('invalid rotation; valid values are: 0, 90, -90, 180')
        return_val = ret_obj(None, 1, keyError_fmt.format('main', 'rotation'))
        
    return ret_obj(obj=screen)

In [11]:
def build_plugin_list(config, resolution, cache):
    '''Build a dictionary of configured plugin objects
    
    Args:
        config(dict): configuration dictionary 
        resolution(tuple): X, Y resolution of screen
        cache(obj: Cache): cache object for managing downloads of images
        
    Returns:
        dict of Plugin'''
    # get the expected key-word args from the Plugin() spec
    spec_kwargs = getfullargspec(Plugin).args

    plugins = []

    for section, values in config.items():
        # ignore the other sections
        if section.startswith('Plugin:'):
            logging.info(f'[[ {section} ]]')

            my_config = {}
            # add all the spec_kwargs from the config
            plugin_kwargs = {}
            for key, val in values.items():
                if key in spec_kwargs:
                    my_config[key] = val
                else:
                    # add everything that is not one of the spec_kwargs to this dict
                    plugin_kwargs[key] = val

            # populate the kwargs my_config dict that will be passed to the Plugin() object
            my_config['name'] = section
            my_config['resolution'] = resolution
            my_config['config'] = plugin_kwargs
            my_config['cache'] = cache
            # force layout to one-bit mode for non-HD screens
            my_config['force_onebit'] = config['main']['force_onebit']
            
            try:
                module = import_module(f'{constants.plugins}.{values["plugin"]}')
                my_config['update_function'] = module.update_function
                my_config['layout'] = getattr(module.layout, values['layout'])
            except KeyError as e:
                logging.info('no module specified; skipping update_function and layout')
                continue
            except ModuleNotFoundError as e:
                logging.warning(f'error: {e} while loading module {constants.plugins}.{values["plugin"]}')
                logging.warning(f'skipping plugin')
                continue
            except AttributeError as e:
                logging.warning(f'could not find layout "{my_config["layout"]}" in {my_config["name"]}')
                logging.warning(f'skipping plugin')
                continue
            my_plugin = Plugin(**my_config)
            try:
                my_plugin.update()
            except AttributeError as e:
                logging.warning(f'ignoring plugin {my_plugin.name} due to missing update_function')
                logging.warning(f'plugin threw error: {e}')
                continue
            logging.info(f'appending plugin {my_plugin.name}')
            plugins.append(my_plugin)
            
            
    if len(plugins) < 1:
        my_config = {}
        logging.warning('no plugins were loaded! falling back to default plugin.')
        my_config['name'] = 'default plugin'
        my_config['resolution'] = resolution
        my_config['cache'] = cache
        try:
            module = import_module(f'{constants.plugins}.default')
        except ModuleNotFoundError as e:
            msg = f'could not load {constants.plugins}.default'
            logging.error(msg)
            do_exit(1, msg)
        my_config['update_function'] = module.update_function
        my_config['layout'] = getattr(module.layout, 'default')
        my_plugin = Plugin(**my_config)
        plugins.append(my_plugin)
        
    return plugins

In [34]:
def update_loop(plugins, screen, max_refresh=2):
    exit_code = 1
    logging.info('starting update loop')
    
    
    def update_plugins(): 
        '''run through all active plugins and update while recording the priority'''
        my_list = []
        logging.info('*'*15)
        logging.info(f'My PID: {os.getppid()}')
        logging.info(f'updating {len(plugins)} plugins')
        for plugin in plugins:
            plugin.update()
            logging.debug(f'update: [{plugin.name}]-p: {plugin.priority}')
            my_list.append(plugin.priority)
        logging.debug(f'priorities: {my_list}')
        return my_list
    
    def write_display(refresh_count):
        logging.debug('writing image to screen')
        try:
            screen.writeEPD(this_plugin.image)
            refresh_count += 1
        except ScreenError as e:
            logging.critical(f'"{this_plugin.name}" returned bad image data; could not update screen')
    
        
    # use itertools cycle to move between list elements
    plugin_cycle = cycle(plugins)
    plugin_is_active = False
    # current plugin for display
    this_plugin = next(plugin_cycle)
    # track time plugin is displayed for
    this_plugin_timer = Update()
    # each plugin generates a unique hash whenever it is updated
    this_hash = ''
    
    # update all the plugins and record priority
    priority_list = update_plugins()
    # this var name is confusing -- it's actually the lowest number to indicate **maximum** priority
    max_priority = min(priority_list)
    # record for comparison
    last_priority = max_priority
    
    # count the number of refreshes for HD Screens
    refresh_count = 0    
    
    logging.info(f'max_priority: {max_priority}')
    
  

    
    # find the first active plugin and update immediately
    update_plugins()
    logging.debug('#### Writing first plugin')
    for plugin in plugins:
        if plugin.priority <= max_priority:
            this_hash = plugin.hash
            this_plugin = plugin
            break
    write_display(refresh_count)
    
#             logging.info(f'**** displaying {plugin.name} ****')
# #             screen.initEPD()
#             try:
#                 screen.writeEPD(plugin.image)
#             except ScreenError as e:
#                 logging.critical(f'could not write to EPD: {e}')
    try:
        screen.writeEPD(this_plugin.image)
    except ScreenError as e:
        logging.critical('')
    
    

    # loop for updating plugins
    with InterruptHandler() as h:
        while True:    
            if h.interrupted:
                logging.info('caught interrupt -- stoping execution')
                exit_code = 0
                break

            priority_list = update_plugins()

            # priority increases as it gets lower; 0 is considered the bottom,
            # but some modules may temporarily have a negative priority to indicate a critical
            # update 
            last_priority = max_priority
            max_priority = min(priority_list)
            
            logging.debug(f'{this_plugin.name}: last updated: {this_plugin_timer.last_updated}, min_display_time: {this_plugin.min_display_time}')
            
            
            # if the timer has expired OR a module has changed the priority setting begin the update procedure
            if this_plugin_timer.last_updated > this_plugin.min_display_time or max_priority < last_priority:
                logging.info(f'plugin expired -- switching plugin')
                plugin_is_active = False
                
                # cycle through plugins, looking for the next plugin that has high priority
                while not plugin_is_active:
                    this_plugin = next(plugin_cycle)
                    logging.debug(f'checking priority of {this_plugin.name}')
                    if this_plugin.priority <= max_priority:
                        plugin_is_active = True
                    else:
                        logging.debug('trying next plugin')
                        pluggin_is_active = False
                        
                logging.info(f'displaying {this_plugin.name} -- priority: {this_plugin.priority}/{max_priority}')
                
                if this_hash != this_plugin.hash:
                    logging.debug('data refreshed, refreshing screen')
                    this_hash = this_plugin.hash
#                     screen.initEPD()
                    logging.debug(f'image type: {type(this_plugin.image)}')
    
                    # wipe screen if the max_refresh count is exceeded
                    if refresh_count > max_refresh:
                        refresh_count = 0
                        if screen.HD:
                            logging.info('max_refresh exceeded, wiping screen prior to next update')
                            screen.clearEPD()
                        else:
                            logging.debug(f'{max_refresh - refresh_count} refreshes remain before full wipe')
                    
                    write_display(refresh_count)
#                     logging.debug('writing image to screen')
#                     try:
#                         screen.writeEPD(this_plugin.image)
#                         refresh_count += 1
#                     except ScreenError as e:
#                         logging.critical(f'"{this_plugin.name}" returned bad image data; could not update screen')

#                     if screen.writeEPD(this_plugin.image):
#                         logging.debug('successfully wrote image')
#                         refresh_count += 1
#                     else:
#                         logging.warning('#=#=# failed to write image #=#=#')
#                         logging.info('trying next plugin')
#                         plugin_is_active = False
                        
                    
                else:
                    logging.debug('plugin data not refreshed -- skipping screen refresh')
                this_plugin_timer.update()    
                    
        
            sleep(1)
    return exit_code

In [35]:
def main():
    
    # change the working directory -- this simplifies all path work later on
    os.chdir(constants.absolute_path)
    
    # set the absolute path to the current directory
    absolute_path = constants.absolute_path
       
    # set up logging
    logging.config.fileConfig(constants.logging_config)
    logger = logging.getLogger(__name__)
    
    # get command line and config file arguments
    cmd_args = get_cmd_line_args()
    
    if hasattr(cmd_args, 'unknown'):
        print(f'Unknown arguments: {cmd_args.unknown}\n\n')
        cmd_args.parser.print_help()
        return
        
    
    config_files = get_config_files(cmd_args)
    
    # merge file and commandline (right-most over-writes left)
    config = ArgConfigParse.merge_dict(config_files.config_dict, cmd_args.nested_opts_dict)
    
    if cmd_args.options.version:
        print(constants.version_string)
        return
    
    if cmd_args.options.plugin_info:
        get_help.get_help(cmd_args.options.plugin_info)
        return
    
    if cmd_args.options.list_plugins:
        get_help.get_help()
        return
    
    if cmd_args.options.run_plugin_func:
        run_module.run_module(cmd_args.options.run_plugin_func)
        return
    
    # make sure all the integer-like strings are converted into integers
    config = sanitize_vals(config)
#     return config
    
    
    logger.setLevel(config['main']['log_level'])
    logging.root.setLevel(config['main']['log_level'])
    
    logging.info(f'********** PaperPi {constants.version} Starting **********')
    
    # configure screen
    screen_return = setup_display(config)

    if screen_return['obj']:
        screen = screen_return['obj']
    else:
        clean_up(None, None)
        logging.error(f'config files used: {config_files.config_files}')
        do_exit(**screen_return)
    
    # try to set up the splash screen several times here -- this may solve the None image problem.
    splash = setup_splash(config, screen.resolution)
    
    if splash:
        logging.debug('displaying splash screen')
        logging.debug(f'image type: {type(splash.image)}')
        try:
            screen.writeEPD(splash.image)
        except ScreenError as e:
            logging.critical(f'Could not write to EPD: {e}')
    
    
    cache = CacheFiles(path_prefix=constants.app_name)
    
    # force one bit mode if screen is NOT HD
    if screen.HD:
        config['main']['force_onebit'] = False
    else:
        config['main']['force_onebit'] = True

    plugins = build_plugin_list(config=config, resolution=screen.resolution, cache=cache)
    
    exit_code = update_loop(plugins, screen)

    logging.info('caught terminate signal -- cleaning up and exiting')
    clean_up(cache, screen)
    
    return exit_code

In [36]:
if __name__ == "__main__":
    # remove jupyter runtime junk for testing
    if len(sys.argv) >= 2 and 'ipykernel' in sys.argv[0]:
        sys.argv = [sys.argv[0]]
        sys.argv.extend(sys.argv[3:])
    exit_code = main()
    sys.exit(exit_code)

15:39:49 1613195952:main:51:INFO - ********** PaperPi 0.2.08_writeEPD_crash Starting **********
15:39:49 Screen:_load_hd:395:DEBUG - configuring IT8951 epd
15:39:51 Screen:epd:330:DEBUG - epd configuration {'epd': <IT8951.display.AutoEPDDisplay object at 0xadade810>, 'resolution': [1200, 825], 'clear_args': {}, 'one_bit_display': False, 'constants': <module 'IT8951.constants' from '/home/pi/.local/share/virtualenvs/epd_display-ApAYs8Kw/lib/python3.7/site-packages/IT8951/constants.py'>, 'mode': 'L'}
15:39:51 Screen:rotation:377:DEBUG - rotation=0, resolution=[1200, 825]
15:39:51 Screen:rotation:377:DEBUG - rotation=0, resolution=[1200, 825]
15:39:51 3124529230:setup_splash:2:DEBUG - checking splash settings
15:39:51 3124529230:setup_splash:4:DEBUG - checking splash screen settings
15:39:51 3124529230:setup_splash:6:DEBUG - splash enabled in confg file
15:39:51 3124529230:setup_splash:15:DEBUG - splash screen enabled
15:39:51 3124529230:setup_splash:23:DEBUG - configuring splash screen: 

15:39:52 Block:_text2image:1015:DEBUG - pasting hcenterd
15:39:52 Block:_text2image:1019:DEBUG - pasting vcentered
15:39:52 Block:_text2image:1022:DEBUG - paste coordinates: (385, 16)
15:39:52 Block:_text_formatter:935:DEBUG - formatting string: https://github.com/ txoof/epd_display
15:39:52 Block:_text2image:975:DEBUG - text size: (732, 188)
15:39:52 Block:_text2image:1015:DEBUG - pasting hcenterd
15:39:52 Block:_text2image:1019:DEBUG - pasting vcentered
15:39:52 Block:_text2image:1022:DEBUG - paste coordinates: (234, 30)
15:39:52 3124529230:setup_splash:26:DEBUG - splash screen image type: <class 'PIL.Image.Image'>
15:39:52 1613195952:main:67:DEBUG - displaying splash screen
15:39:52 1613195952:main:68:DEBUG - image type: <class 'PIL.Image.Image'>
15:39:52 Screen:wrapper:246:DEBUG - initing display
15:39:52 Screen:wrapper:268:DEBUG - HD display
15:39:52 Screen:_full_writeEPD_hd:571:DEBUG - writing to display using GC16 (full display update)
15:39:52 Screen:wrapper:275:DEBUG - sleepin

15:39:56 Layout:_calculate_layout:197:DEBUG - width: 1, height: 0.1
15:39:56 Layout:_scale_font:240:DEBUG - scaling font size
15:39:56 Layout:_scale_font:260:DEBUG - y target size reached
15:39:56 Layout:_scale_font:263:DEBUG - calculated font size: 48
15:39:56 Layout:_calculate_layout:232:DEBUG - absolute coordinates provided
15:39:56 Layout:_calculate_layout:234:DEBUG - block coordinates: (0, 742)
15:39:56 Layout:_set_blocks:112:INFO - [[____setting blocks____]]
15:39:56 Layout:_set_blocks:117:INFO - section: [___________wordtime___________]
15:39:56 Layout:_set_blocks:123:DEBUG - setting block type: TextBlock
15:39:56 Block:area:293:DEBUG - block area: (1200, 742)
15:39:56 Block:padding:320:DEBUG - padded area: [1190, 732]
15:39:56 Block:inverse:378:DEBUG - fill: 255, bkground: 0
15:39:56 Block:__init__:181:DEBUG - creating Block
15:39:56 Block:font:810:DEBUG - setting old_font = None
15:39:56 Block:_calc_maxchar:904:DEBUG - calculating maximum characters for font ('Anton', 'Regular

15:39:57 Layout:_set_blocks:117:INFO - section: [____________title_____________]
15:39:57 Layout:_set_blocks:123:DEBUG - setting block type: TextBlock
15:39:57 Block:area:293:DEBUG - block area: (1200, 412)
15:39:57 Block:padding:320:DEBUG - padded area: [1190, 402]
15:39:57 Block:inverse:378:DEBUG - fill: 0, bkground: 255
15:39:57 Block:__init__:181:DEBUG - creating Block
15:39:57 Block:font:810:DEBUG - setting old_font = None
15:39:57 Block:_calc_maxchar:904:DEBUG - calculating maximum characters for font ('Oswald', 'Medium')
15:39:57 Block:_calc_maxchar:916:DEBUG - average character width: 67.24064711830131
15:39:57 Block:_calc_maxchar:919:DEBUG - maximum characters per line: 18
15:39:57 Block:_text_formatter:935:DEBUG - formatting string: NONE
15:39:57 Block:_text2image:975:DEBUG - text size: (302, 160)
15:39:57 Block:_text2image:1015:DEBUG - pasting hcenterd
15:39:57 Block:_text2image:1019:DEBUG - pasting vcentered
15:39:57 Block:_text2image:1022:DEBUG - paste coordinates: (449, 1

15:40:03 Block:_text2image:975:DEBUG - text size: (93, 29)
15:40:03 Block:_text2image:1019:DEBUG - pasting vcentered
15:40:03 Block:_text2image:1022:DEBUG - paste coordinates: (5, 26)
15:40:03 Layout:_set_blocks:117:INFO - section: [____________title_____________]
15:40:03 Layout:_set_blocks:123:DEBUG - setting block type: TextBlock
15:40:03 Block:area:293:DEBUG - block area: (1200, 412)
15:40:03 Block:padding:320:DEBUG - padded area: [1190, 402]
15:40:03 Block:inverse:378:DEBUG - fill: 0, bkground: 255
15:40:03 Block:__init__:181:DEBUG - creating Block
15:40:03 Block:font:810:DEBUG - setting old_font = None
15:40:03 Block:_calc_maxchar:904:DEBUG - calculating maximum characters for font ('Oswald', 'Medium')
15:40:04 Block:_calc_maxchar:916:DEBUG - average character width: 67.24064711830131
15:40:04 Block:_calc_maxchar:919:DEBUG - maximum characters per line: 18
15:40:04 Block:_text_formatter:935:DEBUG - formatting string: NONE
15:40:04 Block:_text2image:975:DEBUG - text size: (302, 16

...

15:50:58 3899685089:update_plugins:14:DEBUG - update: [Plugin: LibreSpot]-p: 32768
15:50:58 Plugin:_is_ready:230:DEBUG - throttling in effect -- wait for 248.55754241199975 seconds before requesting update
15:50:58 3899685089:update_plugins:14:DEBUG - update: [Plugin: Weather Den Haag]-p: 2
15:50:58 3899685089:update_plugins:16:DEBUG - priorities: [2, 2, 32768, 32768, 2]
15:50:58 3899685089:update_loop:92:DEBUG - Plugin: Weather Den Haag: last updated: 12.213588646000062, min_display_time: 20
15:50:59 3899685089:update_plugins:9:INFO - ***************
15:50:59 3899685089:update_plugins:10:INFO - My PID: 854
15:50:59 3899685089:update_plugins:11:INFO - updating 5 plugins
15:50:59 Plugin:_is_ready:230:DEBUG - throttling in effect -- wait for 2.784090238999852 seconds before requesting update
15:50:59 3899685089:update_plugins:14:DEBUG - update: [Plugin: New Yorker Comic]-p: 2
15:50:59 Plugin:_is_ready:230:DEBUG - throttling in effect -- wait for 3.200115726999684 seconds before requestin

15:51:03 3899685089:update_plugins:9:INFO - ***************
15:51:03 3899685089:update_plugins:10:INFO - My PID: 854
15:51:04 3899685089:update_plugins:11:INFO - updating 5 plugins
15:51:04 Plugin:_is_ready:230:DEBUG - throttling in effect -- wait for 13.462670482999783 seconds before requesting update
15:51:04 3899685089:update_plugins:14:DEBUG - update: [Plugin: New Yorker Comic]-p: 2
15:51:04 Plugin:_is_ready:230:DEBUG - throttling in effect -- wait for 13.912763584999993 seconds before requesting update
15:51:04 3899685089:update_plugins:14:DEBUG - update: [Plugin: Word Clock]-p: 2
15:51:04 lms_client:update_function:104:DEBUG - update_function for plugin Plugin: LMS MacPlay, version 0.1.2
15:51:04 lms_client:update_function:156:WARNING - could not get now playing information for "MacPlay": ValueError invalid value "None" for "player_id"
15:51:04 lms_client:update_function:157:WARNING - check player_name in config file. Is "MacPlay" connected to the LMS server?
15:51:04 3899685089:

15:51:09 3899685089:update_plugins:14:DEBUG - update: [Plugin: LibreSpot]-p: 32768
15:51:09 Plugin:_is_ready:230:DEBUG - throttling in effect -- wait for 237.40500870399956 seconds before requesting update
15:51:09 3899685089:update_plugins:14:DEBUG - update: [Plugin: Weather Den Haag]-p: 2
15:51:09 3899685089:update_plugins:16:DEBUG - priorities: [2, 2, 32768, 32768, 2]
15:51:09 3899685089:update_loop:92:DEBUG - Plugin: New Yorker Comic: last updated: 2.1814722859999165, min_display_time: 15
15:51:10 3899685089:update_plugins:9:INFO - ***************
15:51:10 3899685089:update_plugins:10:INFO - My PID: 854
15:51:10 3899685089:update_plugins:11:INFO - updating 5 plugins
15:51:10 Plugin:_is_ready:230:DEBUG - throttling in effect -- wait for 7.031201064000015 seconds before requesting update
15:51:10 3899685089:update_plugins:14:DEBUG - update: [Plugin: New Yorker Comic]-p: 2
15:51:10 Plugin:_is_ready:230:DEBUG - throttling in effect -- wait for 7.477469277999262 seconds before requestin

15:51:15 Plugin:_is_ready:230:DEBUG - throttling in effect -- wait for 2.219105918999958 seconds before requesting update
15:51:15 3899685089:update_plugins:14:DEBUG - update: [Plugin: Word Clock]-p: 2
15:51:15 Plugin:_is_ready:230:DEBUG - throttling in effect -- wait for 3.9221434860000954 seconds before requesting update
15:51:15 3899685089:update_plugins:14:DEBUG - update: [Plugin: LMS MacPlay]-p: 32768
15:51:15 Plugin:_is_ready:230:DEBUG - throttling in effect -- wait for 2.553003183999863 seconds before requesting update
15:51:15 3899685089:update_plugins:14:DEBUG - update: [Plugin: LibreSpot]-p: 32768
15:51:15 Plugin:_is_ready:230:DEBUG - throttling in effect -- wait for 231.09699368899965 seconds before requesting update
15:51:15 3899685089:update_plugins:14:DEBUG - update: [Plugin: Weather Den Haag]-p: 2
15:51:15 3899685089:update_plugins:16:DEBUG - priorities: [2, 2, 32768, 32768, 2]
15:51:15 3899685089:update_loop:92:DEBUG - Plugin: New Yorker Comic: last updated: 8.491416503

SystemExit: 0

In [37]:
!jupyter-nbconvert --to python --template python_clean paperpi.ipynb

[NbConvertApp] Converting notebook paperpi.ipynb to python
[NbConvertApp] Writing 23636 bytes to paperpi.py


In [ ]:
# logger = logging.getLogger(__name__)
# logger.root.setLevel('DEBUG')